In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import svm
from sklearn.semi_supervised import label_propagation

rng = np.random.RandomState(0)

iris = datasets.load_iris()

X = iris.data[:, :2]
y = iris.target

# step size in the mesh
h = .02

y_30 = np.copy(y)
y_30[rng.rand(len(y)) < 0.3] = -1
y_50 = np.copy(y)
y_50[rng.rand(len(y)) < 0.5] = -1
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
    ls30 = (label_propagation.LabelSpreading().fit(X, y_30),
        y_30)
ls50 = (label_propagation.LabelSpreading().fit(X, y_50),
        y_50)
ls100 = (label_propagation.LabelSpreading().fit(X, y), y)
rbf_svc = (svm.SVC(kernel='rbf').fit(X, y), y)

# create a mesh to plot in
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# title for the plots
titles = ['Label Spreading 30% data',
          'Label Spreading 50% data',
          'Label Spreading 100% data',
          'SVC with rbf kernel']

color_map = {-1: (1, 1, 1), 0: (0, 0, .9), 1: (1, 0, 0), 2: (.8, .6, 0)}

for i, (clf, y_train) in enumerate((ls30, ls50, ls100, rbf_svc)):
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    plt.subplot(2, 2, i + 1)
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired)
    plt.axis('off')

    # Plot also the training points
    colors = [color_map[y] for y in y_train]
    plt.scatter(X[:, 0], X[:, 1], c=colors, edgecolors='black')

    plt.title(titles[i])

plt.suptitle("Unlabeled points are colored white", y=0.1)
plt.show()

In [ ]:
import sys
import os
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import setup

is_windows = sys.platform.startswith('win')
sep = '\\'

if is_windows == False:
    sep = '/'

path = os.getcwd()+sep+'data'+sep

#sinthetic
X, Y, description = setup.load2CHT(path, sep)
X1 = X[0:200]
X2 = X[1000:1200]


def solve(m1,m2,s1,s2):
    x1 = (s1*s2*np.sqrt((-2*np.log(s1/s2)*s2**2)+2*s1**2*np.log(s1/s2)+m2**2-2*m1*m2+m1**2)+m1*s2**2-m2*s1**2)/(s2**2-s1**2)
    x2 = -(s1*s2*np.sqrt((-2*np.log(s1/s2)*s2**2)+2*s1**2*np.log(s1/s2)+m2**2-2*m1*m2+m1**2)-m1*s2**2+m2*s1**2)/(s2**2-s1**2)
    return x1,x2


def ovl(x, x2):
    m1 = np.mean(x)
    std1 = np.std(x)
    m2 = np.mean(x2)
    std2 = np.std(x2)

    r = solve(m1,m2,std1,std2)[0]
    print(r)
    if np.min(x) < np.min(x2):
        #print("D1 < D2")
        indX = x[:,0]>r
        indX2 = x2[:,0]<r
    else:
        #print("D2 < D1")
        indX = x[:,0]<r
        indX2 = x2[:,0]>r

    
    return len(indX), len(indX2)


print(ovl(X1, X2))

In [ ]:
import sys
import os
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from sklearn.neighbors.kde import KernelDensity
import numpy as np
from scipy import integrate

is_windows = sys.platform.startswith('win')
sep = '\\'

if is_windows == False:
    sep = '/'

path = os.getcwd()+sep+'data'+sep

#sinthetic
X, Y, description = setup.load2CHT(path, sep)
X1 = X[0:200]
X2 = X[1000:1200]
#print(X1[0])
#print(X2[1199])
xMin = X1[0]
xMax = X2[199]
#kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(np.vstack([X1, X2]))
#kde1 = kde.score_samples(X1)
#kde2 = kde.score_samples(X2)
kde1 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(X1)
kde2 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(X2)

# Calculate overlap between the two KDEs.
def y_pts(pt):
    y_pt = min(kde1.score_samples(pt), kde2.score_samples(pt))
    return y_pt
# Store overlap value.
overlap = integrate.nquad(y_pts, X1) 
print(overlap)

In [ ]:
import sys
import os
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
import matplotlib.pyplot as plt
import sklearn
from scipy import integrate
import numpy as np
is_windows = sys.platform.startswith('win')
sep = '\\'

if is_windows == False:
    sep = '/'

path = os.getcwd()+sep+'data'+sep

#sinthetic
X, Y, description = setup.load2CHT(path, sep)
X1 = X[0:200]
X2 = X[0:200]




sample = np.array(X)
kde1 = sklearn.neighbors.KernelDensity().fit(X1)
kde2 = sklearn.neighbors.KernelDensity().fit(X2)
def f_kde(x,y):
    return min(np.exp((kde1.score_samples([[x,y]]))), np.exp((kde2.score_samples([[x,y]]))))

x1, y1 = X1[0][0], X1[0][1]
integrate.nquad(f_kde, [[-10, x1],[-10, y1]]) #[[-np.inf, x1],[-np.inf, y1]]


In [1]:
import sys
import os
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from source import util
import matplotlib.pyplot as plt
import sklearn
from scipy import integrate
import numpy as np
is_windows = sys.platform.startswith('win')
sep = '\\'

if is_windows == False:
    sep = '/'

path = os.getcwd()+sep+'data'+sep

#sinthetic
data, labels, description = setup.load2CHT(path, sep)
X = data[810:1160]
y = labels[810:1160]
Ut = data[800:1160]
predicted = labels[800:1160]
classes = [0,1]

pastIndexesByClass = util.slicingClusteredData(y, classes)
currentIndexesByClass = util.slicingClusteredData(predicted, classes)
instancesXByClass, instancesUtByClass = util.unifyInstancesByClass(X, y, Ut, predicted, classes)


def OVL_two_random_arr(arr1, arr2, number_bins):
    # Determine the range over which the integration will occur
    min_value = np.min((arr1.min(), arr2.min()))
    max_value = np.min((arr1.max(), arr2.max()))
    # Determine the bin width
    bin_width = (max_value-min_value)/number_bins
    #For each bin, find min frequency
    lower_bound = min_value #Lower bound of the first bin is the min_value of both arrays
    min_arr = np.empty(number_bins) #Array that will collect the min frequency in each bin
    for b in range(number_bins):
        higher_bound = lower_bound + bin_width #Set the higher bound for the bin
        #Determine the share of samples in the interval
        freq_arr1 = np.ma.masked_where((arr1<lower_bound)|(arr1>=higher_bound), arr1).count()/len(arr1)
        freq_arr2 = np.ma.masked_where((arr2<lower_bound)|(arr2>=higher_bound), arr2).count()/len(arr2)
        #Conserve the lower frequency
        min_arr[b] = np.min((freq_arr1, freq_arr2))
        lower_bound = higher_bound #To move to the next range
    return min_arr.sum()  


#arr1 = X1[pastIndexesByClass[0]]
#arr2 = X2[currentIndexesByClass[0]]
#OVL_two_random_arr(arr1, arr2, 160)
res = util.getBhattacharyyaScoresByClass(X, Ut, classes)
print(len(instancesXByClass[0]))
print(len(instancesXByClass[1]))
print(res)

AttributeError: module 'source.util' has no attribute 'getBhattacharyyaScoresByClass'

In [ ]:
import sys
import os
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import setup

is_windows = sys.platform.startswith('win')
sep = '\\'

if is_windows == False:
    sep = '/'

path = os.getcwd()+sep+'data'+sep

#sinthetic
X, Y, description = setup.load2CHT(path, sep)
X = X[0:200]
Y = Y[0:200]
'''    
# we create 40 separable points
np.random.seed(0)
X = np.r_[np.random.randn(20, 2) - [2, 2], np.random.randn(20, 2) + [2, 2]]
Y = [0] * 20 + [1] * 20
'''
# figure number
fignum = 1

# fit the model
for name, penalty in (('unreg', 1), ('reg', 0.05)):

    clf = svm.SVC(kernel='linear', C=penalty)
    clf.fit(X, Y)

    # get the separating hyperplane
    w = clf.coef_[0]
    a = -w[0] / w[1]
    xx = np.linspace(-5, 5)
    yy = a * xx - (clf.intercept_[0]) / w[1]

    # plot the parallels to the separating hyperplane that pass through the
    # support vectors (margin away from hyperplane in direction
    # perpendicular to hyperplane). This is sqrt(1+a^2) away vertically in
    # 2-d.
    margin = 1 / np.sqrt(np.sum(clf.coef_ ** 2))
    yy_down = yy - np.sqrt(1 + a ** 2) * margin
    yy_up = yy + np.sqrt(1 + a ** 2) * margin

    # plot the line, the points, and the nearest vectors to the plane
    plt.figure(fignum, figsize=(4, 3))
    plt.clf()
    plt.plot(xx, yy, 'k-')
    plt.plot(xx, yy_down, 'k--')
    plt.plot(xx, yy_up, 'k--')

    plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=80,
                facecolors='none', zorder=10, edgecolors='y')
    plt.scatter(X[:, 0], X[:, 1], c=Y, zorder=10, cmap=plt.cm.Paired,
                edgecolors='k')

    plt.axis('tight')
    '''
    print(len(clf.support_), clf.support_)
    
    print("X= ",len(X), X)
    #points = np.delete(X, clf.support_)
    dif = [i for i in range(0,len(X))]
    dif = np.setdiff1d(dif, clf.support_)
    points = X[dif]
    print("new X= ",len(points), points)
    '''
    dif = [i for i in range(0,len(X))]
    dif = np.setdiff1d(dif, clf.support_)
    points = X[dif]
    
    plt.scatter(points[:, 0], points[:, 1], c=Y[dif], zorder=10, cmap=plt.cm.Paired,
                edgecolors='r')
    
    x_min = -10
    x_max = 10
    y_min = -10
    y_max = 10
    
    XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]
    Z = clf.predict(np.c_[XX.ravel(), YY.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(XX.shape)
    plt.figure(fignum, figsize=(4, 3))
    plt.pcolormesh(XX, YY, Z, cmap=plt.cm.Paired)

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    plt.xticks(())
    plt.yticks(())
    fignum = fignum + 1

plt.show()



#np.setdiff1d(arr1, arr2)